In [1]:
cd E:/GitHub/computervision

E:\GitHub\computervision


In [2]:
import torch
import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

from torch.nn import CrossEntropyLoss
from datetime import datetime
from torch.optim import lr_scheduler, SGD, Adamax
from modules.training_util import PytorchDataset, PytorchTraining, EarlyStopping
from modules.models_util import DenseNet

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING_DIR = None
# CONTINUE_TRAINING_DIR = 'notebooks/densenet/models/'

MODEL_WEIGHTS_PATH = None

if CONTINUE_TRAINING_DIR:
    MODEL_WEIGHTS_PATH = os.path.join(CONTINUE_TRAINING_DIR,'epoch_15.pth')

# CONFIG VARIABLES
FULLY_TRAIN = True
base_path = 'E:/Datasets/cor-splits'
DATA_DIR = [
    os.path.join(base_path,'sgkf-8-1-1-size-540'),
    os.path.join(base_path,'balanced_train'),
    os.path.join(base_path,'imbalanced_train'),
    os.path.join(base_path,'full_imbalanced_train'),
    os.path.join(base_path,'natural_imbalanced_train')
    ]
START_EPOCH = 1 # Change it from 1 if you're continuing an existing run
N_EPOCHS = 50
BATCH_SIZE = 32
SAVE_INTERVAL = 2 # How many epochs between each backup
## Learning Rate Scheduler
LR = 0.001  # When continuing a existing run, update it to the last epoch LR value
LR_EPOCH_SCHEDULE = 10  # Number of epochs before altering LR
LR_GAMMA = 0.1
## Early Stopping
PATIENCE = LR_EPOCH_SCHEDULE + 1
MIN_DELTA = 0
MIN_EPOCH = 10 # Number of spochs before starting patience counter

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
archs = ['121','161','169']

In [5]:
for arch in archs:
    for dir in DATA_DIR:
        model_arch = DenseNet(architecture=arch)
        
        model = model_arch.load(MODEL_WEIGHTS_PATH, FULLY_TRAIN)
        model = model.to(device)
        
        criterion = CrossEntropyLoss()

        if FULLY_TRAIN:
            params = model.parameters()
        else:
            params = model.heads.parameters()
            
        # optimizer = optim.SGD(model.classifier.parameters(), lr=LR)
        optimizer = SGD(params, lr=LR)
        # optimizer = Adamax(model.parameters(), lr=LR)

        step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)
        es = EarlyStopping(patience=PATIENCE, delta=MIN_DELTA, min_epoch=MIN_EPOCH)
        dataset = PytorchDataset(dir, model_arch.data_transforms, BATCH_SIZE)

        working_folder = 'notebooks/densenet'
        models_folder = 'models'

        curr_time = datetime.now()
        curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

        output_folder = os.path.join(working_folder, models_folder)
        output_folder = os.path.join(output_folder, curr_time)
        
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        trainer = PytorchTraining(device, dataset, output_folder)
        model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, es, START_EPOCH, N_EPOCHS, SAVE_INTERVAL, arch)

c:\Users\Adm\anaconda3\envs\computervision\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


All parameters for model DenseNet requires grad.
Saving model to folder notebooks/densenet\models\2024-02-22 11-10-06

Model DenseNet 121
Fully Trained = True
Dataset E:/Datasets/cor-splits/sgkf-8-1-1-size-540
Learning Rate Epoch Schedule = 10
Learning Rate Gamma = 0.1
Optimizer = SGD
Batch Size = 32


Epoch 1/50
----------

train Loss: 0.7162 Acc: 0.5069
val Loss: 0.7138 Acc: 0.4444
Epoch duration: 0 m 14s
Learning Rate = 0.001



Epoch 2/50
----------

train Loss: 0.6818 Acc: 0.5579
val Loss: 0.7155 Acc: 0.4444
Epoch duration: 0 m 9s
Learning Rate = 0.001



Epoch 3/50
----------



KeyboardInterrupt: 